# Image Study
This notebook is a study of how images are loaded into python, compared with values from MATLAB's imread() function.

In [2]:
# import packages
# Import packages
import ast
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from os import listdir
from os.path import join, normpath
import pandas as pd
import PIL
from scipy.signal import convolve2d
from sys import exit
from tqdm import tqdm

# import local modules
from shapes import Point, Rectangle
from utils import roc_curve, load_roi, search_data, dark_correct, load_roi

In [3]:
# path to test data from MATLAB output
_TEST_PATH = normpath(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works"
    + r"\Customers\DSTL\S3 Kestrel Vision\Task 3 MOE Imager Testing\3.2 External Testing"
    + r"\Analysis\Visible\scene031\matlab"
)

# paths to selected files in test data
_TEST_DARK_PATH = {
    'raw': join(_TEST_PATH, 'DarkMOE.txt'),
    'double': join(_TEST_PATH, 'DarkMOE_double.txt'),
    'subsampled': join(_TEST_PATH, 'DarkMOE_subsampled.txt')
}
_TEST_ND_PATH = {
    'raw': join(_TEST_PATH, 'NE03B.txt'),
    'double': join(_TEST_PATH, 'NE03B_double.txt'),
    'subsampled': join(_TEST_PATH, 'NE03B_subsampled.txt'),
    'sub_dark': join(_TEST_PATH, 'NE03B_subsampled_darkcorrected.txt')
}
_TEST_MOE_1_PATH = {
    'raw': join(_TEST_PATH, 'MOE1.txt'),
    'double': join(_TEST_PATH, 'MOE1_double.txt'),
    'subsampled': join(_TEST_PATH, 'MOE1_subsampled.txt'),
    'sub_dark': join(_TEST_PATH, 'MOE1_subsampled_darkcorrected.txt'),
    'sub_ratio': join(_TEST_PATH, 'MOE1_subsampled_darkcorrected_div_ND.txt')
}

# path to dataset
_DATA_PATH = normpath(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works"
    + r"\Customers\DSTL\S3 Kestrel Vision\Task 3 MOE Imager Testing\3.2 External Testing"
    + r"\VIS\scene031\MOE\set006"
)

_ROI_PATH = normpath(
    r"C:\Users\hsmith\OneDrive - THORLABS Inc\Documents - Thorlabs Spectral Works"
    + r"\Customers\DSTL\S3 Kestrel Vision\Task 3 MOE Imager Testing\3.2 External Testing"
    + r"\Analysis\Visible\scene031\matlab\roi.txt"
)

# paths to selected images in dataset
_DARK_PATH = join(_DATA_PATH, '20220207_101953_DARK.tif')
_ND_PATH = join(_DATA_PATH, '20220207_101953_NE03B.tif')
_MOE_1_PATH = join(_DATA_PATH, '20220207_101953_VIS_MOE_01-04.tif')



### TEST 1
Compare MATLAB imread() with cv.imread(). OpenCv imread() is called using `-1` so that the image is read "as-is".

In [4]:
# read in test darkfield image
test_dark = []
with open(_TEST_DARK_PATH['raw'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_dark.append(
            list(ast.literal_eval(line))
        )
test_dark = np.asarray(test_dark, dtype=np.uint16)
print(
    f'Test Dark: {type(test_dark)} | {np.shape(test_dark)} | {test_dark.dtype}'
    + f' | {np.min(test_dark)} | {np.max(test_dark)}'
)

# read in dataset darkfield image
data_dark = cv.imread(_DARK_PATH, -1)
print(
    f'Data Dark: {type(test_dark)} | {np.shape(test_dark)} | {data_dark.dtype}'
    + f' | {np.min(data_dark)} | {np.max(data_dark)}'
)


Reading test file..
Test Dark: <class 'numpy.ndarray'> | (2048, 2448) | uint16 | 1 | 368
Data Dark: <class 'numpy.ndarray'> | (2048, 2448) | uint16 | 1 | 368


In [5]:
# make assertions about the raw dark data
assert type(test_dark) == type(test_dark)  # assert array types match
assert test_dark.dtype == data_dark.dtype  # assert datatypes match
assert np.shape(test_dark) == np.shape(test_dark)  # assert shapes
assert np.min(test_dark) == np.min(data_dark)  # assert min value
assert np.max(test_dark) == np.max(data_dark)  # assert max value
np.testing.assert_array_almost_equal(test_dark, data_dark, decimal=14)  # assert equal to E-14
print('Darkfield raw data is equivalent. Test 1 passed.')

Darkfield raw data is equivalent. Test 1 passed.


### TEST 2
Convert the opencv dark field image array to floating point, and compare with 'double' values from MATLAB.

In [6]:
# print floating point information
print(np.finfo(np.float64))

# load the floating point test darkfield image
test_dark_float = []
with open(_TEST_DARK_PATH['double'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_dark_float.append(
            list(ast.literal_eval(line))
        )
test_dark_float = np.asarray(test_dark_float)
print(
    f'Test Dark - Float: {type(test_dark_float)} | {np.shape(test_dark_float)} | {test_dark_float.dtype}'
    + f' | {np.min(test_dark_float)} | {np.max(test_dark_float)}'
)

# convert cv dark image to floating point
data_dark_float = data_dark.astype(np.float64) / 65535.0  # <----- numbers must be scaled to 16-bits
print(
    f'Data Dark - Float: {type(data_dark_float)} | {np.shape(data_dark_float)} | {data_dark_float.dtype}'
    + f' | {np.min(data_dark_float)} | {np.max(data_dark_float)}'
)

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
smallest_normal = 2.2250738585072014e-308   smallest_subnormal = 4.9406564584124654e-324
---------------------------------------------------------------

Reading test file..
Test Dark - Float: <class 'numpy.ndarray'> | (2048, 2448) | float64 | 1.5259e-05 | 0.0056153
Data Dark - Float: <class 'numpy.ndarray'> | (2048, 2448) | float64 | 1.5259021896696422e-05 | 0.005615320057984283


In [7]:
# make assertions about the converted dark data
assert type(test_dark_float) == type(test_dark_float)  # assert array types match
assert test_dark_float.dtype == data_dark_float.dtype  # assert datatypes match
assert np.shape(test_dark_float) == np.shape(test_dark_float)  # assert shapes
np.testing.assert_almost_equal(np.min(test_dark_float), np.min(data_dark_float), decimal=10)  # assert min value
np.testing.assert_almost_equal(np.max(test_dark_float), np.max(data_dark_float), decimal=7)  # assert max value
np.testing.assert_array_almost_equal(test_dark_float, data_dark_float, decimal=7)  # assert equal to E-7
print('Darkfield floating point data is equivalent. Test 2 passed.')

Darkfield floating point data is equivalent. Test 2 passed.


### TEST 3
Subsample the test darkfield image and the dataset darkfield image then compare the results.

In [8]:
# load the subsampled floating point test darkfield image
test_dark_sub = []
with open(_TEST_DARK_PATH['subsampled'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_dark_sub.append(
            list(ast.literal_eval(line))
        )
test_dark_sub = np.asarray(test_dark_sub)
print(
    f'Test Dark - Subsampled: {type(test_dark_sub)} | {np.shape(test_dark_sub)} | {test_dark_sub.dtype}'
    + f' | {np.min(test_dark_sub)} | {np.max(test_dark_sub)}'
)

# convert cv dark image to floating point
h, w = data_dark_float.shape
print(f'Original Data Shape: {data_dark_float.shape}')
data_dark_sub = cv.resize(data_dark_float, (w//2, h//2))
print(
    f'Data Dark - Subsampled: {type(data_dark_sub)} | {np.shape(data_dark_sub)} | {data_dark_sub.dtype}'
    + f' | {np.min(data_dark_sub)} | {np.max(data_dark_sub)}'
)

Reading test file..
Test Dark - Subsampled: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 4.5777e-05 | 0.0014649
Original Data Shape: (2048, 2448)
Data Dark - Subsampled: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 4.5777065690089265e-05 | 0.0014648661020828567


In [9]:
# make assertions about the downsampled dark data
assert type(test_dark_sub) == type(data_dark_sub)  # assert array types match
assert test_dark_sub.dtype == data_dark_sub.dtype  # assert datatypes match
assert np.shape(test_dark_sub) == np.shape(data_dark_sub)  # assert shapes
np.testing.assert_almost_equal(np.min(test_dark_sub), np.min(data_dark_sub), decimal=10)  # assert min value
np.testing.assert_almost_equal(np.max(test_dark_sub), np.max(data_dark_sub), decimal=7)  # assert max value
np.testing.assert_array_almost_equal(test_dark_sub, data_dark_sub, decimal=7)  # assert equal to E-7
print('Darkfield downsampled data is equivalent. Test 3 passed.')

Darkfield downsampled data is equivalent. Test 3 passed.


### TEST 4
Compare MOE_1 subsampled and dark corrected with the test data.

In [10]:
# load in MOE 1 test data for comparison
test_moe_1 = []
with open(_TEST_MOE_1_PATH['raw'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_moe_1.append(
            list(ast.literal_eval(line))
        )
test_moe_1 = np.asarray(test_moe_1)
print(
    f'Test MOE 1 Raw: {type(test_moe_1)} | {np.shape(test_moe_1)} | {test_moe_1.dtype}'
    + f' | {np.min(test_moe_1)} | {np.max(test_moe_1)}'
)

# load in MOE 1 floating point data
test_moe_1_float = []
with open(_TEST_MOE_1_PATH['double'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_moe_1_float.append(
            list(ast.literal_eval(line))
        )
test_moe_1_float = np.asarray(test_moe_1_float)
print(
    f'Test MOE 1 Floating Point: {type(test_moe_1_float)} | {np.shape(test_moe_1_float)} | {test_moe_1_float.dtype}'
    + f' | {np.min(test_moe_1_float)} | {np.max(test_moe_1_float)}'
)

# load in MOE 1 subsampled data
test_moe_1_subsample = []
with open(_TEST_MOE_1_PATH['subsampled'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_moe_1_subsample.append(
            list(ast.literal_eval(line))
        )
test_moe_1_subsample = np.asarray(test_moe_1_subsample)
print(
    f'Test MOE 1 Downsampled: {type(test_moe_1_subsample)} | {np.shape(test_moe_1_subsample)} | {test_moe_1_subsample.dtype}'
    + f' | {np.min(test_moe_1_subsample)} | {np.max(test_moe_1_subsample)}'
)

# load in the dark corrected/downsampled MOE_1 image
test_moe_dark_sub = []
with open(_TEST_MOE_1_PATH['sub_dark'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_moe_dark_sub.append(
            list(ast.literal_eval(line))
        )
test_moe_dark_sub = np.asarray(test_moe_dark_sub)
print(
    f'Test MOE 1 Dark Corrected: {type(test_moe_dark_sub)} | {np.shape(test_moe_dark_sub)} | {test_moe_dark_sub.dtype}'
    + f' | {np.min(test_moe_dark_sub)} | {np.max(test_moe_dark_sub)}'
)

Reading test file..
Test MOE 1 Raw: <class 'numpy.ndarray'> | (2048, 2448) | int32 | 44 | 4094
Reading test file..
Test MOE 1 Floating Point: <class 'numpy.ndarray'> | (2048, 2448) | float64 | 0.0006714 | 0.06247
Reading test file..
Test MOE 1 Downsampled: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.00077058 | 0.06247
Reading test file..
Test MOE 1 Dark Corrected: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.00069047 | 0.062398


In [11]:
# load in the MOE 1 image
moe_1_data_raw = cv.imread(_MOE_1_PATH, -1)
print(
    f'MOE 1 Raw: {type(moe_1_data_raw)} | {np.shape(moe_1_data_raw)} | {moe_1_data_raw.dtype}'
    + f' | {np.min(moe_1_data_raw)} | {np.max(moe_1_data_raw)}'
)

# convert to floating point
moe_1_data_float = moe_1_data_raw.astype(np.float64) / 65535.0  # scale to floating point
print(
    f'MOE 1 Floating Point: {type(moe_1_data_float)} | {np.shape(moe_1_data_float)} | {moe_1_data_float.dtype}'
    + f' | {np.min(moe_1_data_float)} | {np.max(moe_1_data_float)}'
)

# downsample the MOE data
h, w = moe_1_data_float.shape
moe_1_data_down= cv.resize(moe_1_data_float, (w//2, h//2))
print(
    f'MOE 1 Downsampled: {type(moe_1_data_down)} | {np.shape(moe_1_data_down)} | {moe_1_data_down.dtype}'
    + f' | {np.min(moe_1_data_down)} | {np.max(moe_1_data_down)}'
)


# subtract downsampled dark from moe_1 data
moe_1_dark_subtract = np.subtract(moe_1_data_down, data_dark_sub)
print(
    f'MOE 1 Dark Corrected: {type(moe_1_dark_subtract)} | {np.shape(moe_1_dark_subtract)} | {moe_1_dark_subtract.dtype}'
    + f' | {np.min(moe_1_dark_subtract)} | {np.max(moe_1_dark_subtract)}'
)

MOE 1 Raw: <class 'numpy.ndarray'> | (2048, 2448) | uint16 | 44 | 4094
MOE 1 Floating Point: <class 'numpy.ndarray'> | (2048, 2448) | float64 | 0.0006713969634546426 | 0.06247043564507515
MOE 1 Downsampled: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.0007705806057831693 | 0.06247043564507515
MOE 1 Dark Corrected: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.000690470740825513 | 0.06239795529106584


### TEST 5
Check the MOE/ND ratio against the test dataset.

In [12]:
# read in the sub-sampled ND image for comparison
test_nd_filter = []
with open(_TEST_ND_PATH['sub_dark'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_nd_filter.append(
            list(ast.literal_eval(line))
        )
test_nd_filter = np.asarray(test_nd_filter)
print(
    f'Test ND Raw: {type(test_nd_filter)} | {np.shape(test_nd_filter)} | {test_nd_filter.dtype}'
    + f' | {np.min(test_nd_filter)} | {np.max(test_nd_filter)}'
)

# load in the MOE/ND ratio test image
test_moe_nd = []
with open(_TEST_MOE_1_PATH['sub_ratio'], 'r') as f:
    print("Reading test file..")
    for line in f:
        test_moe_nd.append(
            list(ast.literal_eval(line))
        )
test_moe_nd = np.asarray(test_moe_nd)
print(
    f'Test MOE 1 / ND: {type(test_moe_nd)} | {np.shape(test_moe_nd)} | {test_moe_nd.dtype}'
    + f' | {np.min(test_moe_nd)} | {np.max(test_moe_nd)}'
)

Reading test file..
Test ND Raw: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.00034333 | 0.062409
Reading test file..
Test MOE 1 / ND: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.22737 | 4.8426


In [13]:
# load the ND image from dataset
nd_image = cv.imread(_ND_PATH, -1)
print(
    f'ND Image Raw: {type(nd_image)} | {np.shape(nd_image)} | {nd_image.dtype}'
    + f' | {np.min(nd_image)} | {np.max(nd_image)}'
)



nd_image = nd_image.astype(np.float64) / 65535.0  # convert to float
print(
    f'ND Image Floating Point: {type(nd_image)} | {np.shape(nd_image)} | {nd_image.dtype}'
    + f' | {np.min(nd_image)} | {np.max(nd_image)}'
)



h, w = nd_image.shape
nd_image = cv.resize(nd_image, (w//2, h//2))  # downsample
print(
    f'ND Image Downsampled: {type(nd_image)} | {np.shape(nd_image)} | {nd_image.dtype}'
    + f' | {np.min(nd_image)} | {np.max(nd_image)}'
)



nd_image = np.subtract(nd_image, data_dark_sub)  # dark correction
print(
    f'ND Image Dark Corrected: {type(nd_image)} | {np.shape(nd_image)} | {nd_image.dtype}'
    + f' | {np.min(nd_image)} | {np.max(nd_image)}'
)




ND Image Raw: <class 'numpy.ndarray'> | (2048, 2448) | uint16 | 22 | 4094
ND Image Floating Point: <class 'numpy.ndarray'> | (2048, 2448) | float64 | 0.0003356984817273213 | 0.06247043564507515
ND Image Downsampled: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.0004234378576333257 | 0.06247043564507515
ND Image Dark Corrected: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.0003433279926756695 | 0.062409399557488365


In [14]:
# divide the MOE image by the ND image
moe_nd_data = np.divide(moe_1_dark_subtract, nd_image)
print(
    f'MOE / ND Image Data: {type(moe_nd_data)} | {np.shape(moe_nd_data)} | {moe_nd_data.dtype}'
    + f' | {np.min(moe_nd_data)} | {np.max(moe_nd_data)}'
)

MOE / ND Image Data: <class 'numpy.ndarray'> | (1024, 1224) | float64 | 0.22737146994931204 | 4.842603550295858


## Load ROI's
Extract the intensities of the ROI regions from the test and dataset images.


In [17]:
targ, _ = load_roi(_ROI_PATH)

print(f'Target ROI: {targ}')

moe_nd_target_class = []
moe_nd_target_class.append(
    (moe_nd_data[
            targ.top_left.y : targ.bottom_left.y+1,
            targ.top_left.x : targ.top_right.x+1
        ]
    ).flatten()
)

moe_nd_target_class = np.asarray(moe_nd_target_class)
print(
    f'MOE / ND Image Data: {type(moe_nd_target_class)} | {np.shape(moe_nd_target_class)} | {moe_nd_target_class.dtype}'
    + f' | {np.min(moe_nd_target_class)} | {np.max(moe_nd_target_class)}'
)

Target ROI: [(969, 539), (1009, 539), (969, 599), (1009, 599)]
MOE / ND Image Data: <class 'numpy.ndarray'> | (1, 2501) | float64 | 0.9462136717151043 | 1.0041734094199812


In [19]:
# extract the ROI's from the test image
test_moe_nd_target_class = []
test_moe_nd_target_class.append(
    (test_moe_nd[
            targ.top_left.y : targ.bottom_left.y+1,
            targ.top_left.x : targ.top_right.x+1
        ]
    ).flatten()
)

test_moe_nd_target_class = np.asarray(test_moe_nd_target_class)
print(
    f'MOE / ND Image Data: {type(test_moe_nd_target_class)} | {np.shape(test_moe_nd_target_class)} | {test_moe_nd_target_class.dtype}'
    + f' | {np.min(test_moe_nd_target_class)} | {np.max(test_moe_nd_target_class)}'
)

MOE / ND Image Data: <class 'numpy.ndarray'> | (1, 2501) | float64 | 0.94621 | 1.0042


In [23]:
# assert arrays equal
np.testing.assert_array_almost_equal(moe_nd_target_class, test_moe_nd_target_class, decimal=4)